In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import *

In [2]:
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
allImages

['Image14001007-5555.jpg',
 'Image14001010-1133.jpg',
 'Image14001013-4795.jpg',
 'Image14001007-4139.jpg',
 'Image14001011-8881.jpg',
 'Image14001012-8839.jpg',
 'Image14001013-0530.jpg',
 'Image14001010-5143.jpg',
 'Image14001011-0007.jpg',
 'Image14001010-0946.jpg',
 'Image14001013-1376.jpg',
 'Image14001012-1404.jpg',
 'Image14001012-7840.jpg',
 'Image14001010-1245.jpg',
 'Image14001010-3452.jpg',
 'Image14001010-5751.jpg',
 'Image14001007-9031.jpg',
 'Image14001012-4352.jpg',
 'Image14001009-0395.jpg',
 'Image14001012-1564.jpg',
 'Image14001013-5299.jpg',
 'Image14001010-5221.jpg',
 'Image14001011-5553.jpg',
 'Image14001012-6790.jpg',
 'Image14001008-8508.jpg',
 'Image14001009-4326.jpg',
 'Image14001009-6531.jpg',
 'Image14001007-8878.jpg',
 'Image14001010-3693.jpg',
 'Image14001013-0478.jpg',
 'Image14001009-0183.jpg',
 'Image14001008-7835.jpg',
 'Image14001009-5987.jpg',
 'Image14001010-5208.jpg',
 'Image14001007-9554.jpg',
 'Image14001011-0416.jpg',
 'Image14001012-5298.jpg',
 

In [3]:
allImagesList = pd.read_csv('../BigDataSource/Teddy2024/附件2/image_data.csv')
allImagesList.shape

(1000, 1)

In [4]:
import chardet
from modules.models.SBERT import Text2Vec

file_name = '../BigDataSource/Teddy2024/附件2/word_test.csv'
# 首先，读取文件的一部分字节来检测编码
with open(file_name, 'rb') as f:
    raw_data = f.read(5000)
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    
allTextList = pd.read_csv(file_name, encoding=encoding)
allTextList = np.array(allTextList)

text2vec = Text2Vec()
for i in trange(len(allTextList)):
    allTextList[i, 1] = text2vec.text2vec(allTextList[i, 1])
print('Done!')

100%|██████████| 100/100 [00:03<00:00, 30.02it/s]

Done!


In [5]:
# allImagesFeatures = []
# for i in trange(len(allImages)):
#     imageTranfer = Image2tensor()
#     file_name = '../BigDataSource/Teddy2024/附件2/ImageData/' + allImages[i]
#     image = Image.open(file_name)
#     features = imageTranfer.image2tensor(image)
#     allImagesFeatures.append(features)
# allImagesFeatures

In [6]:
from modules.models.ResNet50 import Image2tensor
from PIL import Image

# 这里是执行多线程代码，线性运算过慢，线性预算代码在上面，用时8mins
from concurrent.futures import ThreadPoolExecutor, as_completed

# function
def process_image(inputs):
    imageTranfer, image_name = inputs
    file_name = '../BigDataSource/Teddy2024/附件2/ImageData/' + image_name
    image = Image.open(file_name)
    features = imageTranfer.image2tensor(image)
    return features

# input
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
imageTranfer = Image2tensor()
inputList = [(imageTranfer, imageNames) for imageNames in allImages]

# Execute
allImagesFeatures = []
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_image, inputs) for inputs in inputList]
    for future in tqdm(as_completed(futures), total=len(allImages)):
        allImagesFeatures.append(future.result().reshape(-1, ))
print('Done!')

100%|██████████| 1000/1000 [00:19<00:00, 51.65it/s]

Done!


In [7]:
# 将所有变量转成numpy形式
allImagesFeatures_now = []
for i in range(len(allImagesFeatures)):
    imageFeatures = np.array(allImagesFeatures[i]).reshape(-1, )
    allImagesFeatures_now.append(imageFeatures)
allImagesFeatures = np.array(allImagesFeatures_now)
allImagesFeatures.shape

(1000, 1000)

In [18]:
# 将所有变量转成numpy形式
allTextFeatures_now = []
for i in range(len(allTextList)):
    textFeatures = np.array(allTextList[i, 1]).reshape(-1, )
    allTextFeatures_now.append(textFeatures)
allTextFeatures = np.array(allTextFeatures_now)
allTextFeatures.shape

(100, 384)

In [19]:
from modules.models.Retrieve import LSH, L2Index
from time import time 

d = 64  # 向量维度
nb = 190000  # 数据库大小
nq = 256 * 10  # 查询向量的数量
np.random.seed(1234)  # 保证可复现的结果

# 生成一些随机样本向量
xb = np.random.random((nb, d))
xq = np.random.random((nq, d))

t1 = time()
lsh = LSH(k=3, d=d, nbits=d)
topk_distence, topk_indices = lsh.search_topk_embeds(xb, xq)
t2 = time()
print(f'LSH: {t2 - t1 : .2f}s')

t1 = time()
l2index = L2Index(k=3, d=d)
topk_distence, topk_indices = l2index.search_topk_embeds(xb, xq)
t2 = time()
print(f'L2: {t2 - t1 : .2f}s')

LSH:  0.06s
L2:  0.48s


In [20]:
allTextFeatures.shape, allImagesFeatures.shape

((100, 384), (1000, 1000))